In [ ]:
!pip install pandas scikit-learn
!pip install pymorphy3
!pip install -U pymorphy3-dicts-ru
!pip install umap-learn
!pip install seaborn

In [ ]:
import os
import json
import pandas as pd
import pymorphy3
from umap import UMAP
import seaborn as sns
import numpy as np

In [ ]:
CENSORED_MODE = True # установите False, чтобы снять защиту цензуры

BASE_COLUMN = 'text' if CENSORED_MODE else 'censored'

In [ ]:
files = os.listdir('processed')

stop_words = []
with open('./stop_words.txt') as f:
    stop_words = [item.replace('\n', '') for item in f.readlines()]
stop_words = set(stop_words)

In [ ]:
# функция чтения .json файла и преобразования в dict
def get_file_as_dict(name: str) -> dict:
    with open(f'./dataset_raw/{name}', 'r') as f:
        return json.loads(f.read())

dataset = [get_file_as_dict(name) for name in files if '.json' in name ] # преобразуем данные в 
len(dataset)

In [ ]:
print(dataset[0][BASE_COLUMN][:100]) # первые 100 символов из текста

In [ ]:
import re

symbols_to_drop = set(('(', ')', '.', ',', '?', '!', '«', '»', '—', '-', '’', '…', '(', ')', '’', '–', '"'))


def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

for item in dataset:
    item['original'] = item['text']
    item['original_censored'] = item['censored']
    # Переводим все в lowercase и удаляем лишние пробелы
    item['title'] = item['title'].lower().strip()
    item['text'] = item['text'].lower().strip()
    item['censored'] = item['censored'].lower().strip()

    # уничтожаем спец символы в результате парсинга
    item['text'] = item['text'].replace('\xa0', ' ')
    item['censored'] = item['censored'].replace('\xa0', ' ')

    item['text'] = item['text'].replace('\u2005', ' ')
    item['censored'] = item['censored'].replace('\u2005', ' ')

    item['text'] = item['text'].replace('\n', ' ')
    item['censored'] = item['censored'].replace('\n', ' ')

    # удаляем ненужные символы
    for symbol in symbols_to_drop:
        item['text'] = item['text'].replace(symbol, ' ')
        item['censored'] = item['censored'].replace(symbol, ' ')

    item['text'] = remove_extra_spaces(item['text'])
    item['censored'] = remove_extra_spaces(item['censored'])
    
    # удаляем стоп слова
    item['text'] = ' '.join([word for word in item['text'].split(' ') if word not in stop_words])
    item['censored'] = ' '.join([word for word in item['censored'].split(' ') if word not in stop_words])
    
dataset[0]['censored'][:100] 

In [ ]:
from pprint import pprint
morph = pymorphy3.MorphAnalyzer() # морфологический анализатор текста, если у вас не работает, тогда pymorphy3 необходимо заменить на pymorphy2 (в install, в импортах и так далее. процесс установки итп есть в документации)

pprint(morph.parse('был')[0].normal_form)


In [ ]:
# преобразуем все (леммы, слова, тоекены - это одно и тоже) в нормальную форму (начальная форма, единственное число для каждого типа слова)

for item in dataset:
    item['text_normalized'] = ' '.join([morph.parse(word)[0].normal_form for word in item['text'].split(' ')])
    item['censored_normalized'] = ' '.join([morph.parse(word)[0].normal_form for word in item['censored'].split(' ') if word not in stop_words or len(word) < 2])

dataset[0]['censored_normalized'][:100] # пример НФ



In [ ]:
CENSORED_COLUMNS = ['text', 'text_normalized'] if not CENSORED_MODE else []

df = pd.DataFrame.from_dict(dataset)
columns = df.columns
censoder_mode_columns = ['title', 'censored', 'censored_normalized'] + CENSORED_COLUMNS

df[censoder_mode_columns].head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer


corpus = df.text_normalized.to_list()

count_vectorizer = CountVectorizer()
hashing_vectorizer = HashingVectorizer(n_features=1000)
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
X_count_vectorizer = count_vectorizer.fit_transform(corpus)
X_hashing_vectorizer = hashing_vectorizer.fit_transform(corpus)
X_tfidf_vectorizer = tfidf_vectorizer.fit_transform(corpus)                   

In [ ]:
print(X_count_vectorizer.toarray().shape)
list(X_count_vectorizer.toarray())[:1][0][:600]

In [ ]:
print(X_hashing_vectorizer.toarray().shape)
list(X_hashing_vectorizer.toarray())[:1][0][:600] # это короче вектор из всех слов

In [ ]:
print(X_tfidf_vectorizer.toarray().shape)
list(X_tfidf_vectorizer.toarray())[:1][0][:600]

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import TSNE

X_embedded = UMAP(n_components=10, n_neighbors=4, random_state=42, n_jobs=1).fit_transform(X_tfidf_vectorizer) # для дальнейшей обработки
X_embedded_visual = UMAP(n_components = 2, n_neighbors=4, random_state=42, n_jobs=1).fit_transform(X_tfidf_vectorizer) # для визуализации
X_embedded[:5] # преобразуем наши вектора с длиной 991 элементов, в вектор размером в 20 элементов

In [ ]:
X_embedded_visual[:5]

Попробуем визуализировать и увидеть следующее:

1. группы точек, которые близки друг к другу, указывают на то, что песни в имеют схожую тематику, которая выражается похожим наборов слов

2. Точки, которые далеко от всех остальных,представляют песни, которые уникальны по своему содержанию или стилю (в нашем случае вероятно по языку)

3. Области с большей плотностью точек могут указывать на наиболее общие или популярные слова в корпусе песен.

In [ ]:
sns.scatterplot(x=X_embedded_visual[:, 0], y=X_embedded_visual[:, 1])

In [ ]:
# Инициализация списка для хранения оценок силуэта
kmeans_scores = []

# Перебор различных значений k (от 2 до 10)
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_tfidf_vectorizer)
    score = silhouette_score(X_tfidf_vectorizer, kmeans.labels_)
    kmeans_scores.append(score)

print(kmeans_scores)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, axes = plt.subplots(7, 2, figsize=(12, 30))
axes = axes.ravel()

# Удаляем последний график, так как он не нужен в данной конфигурации
fig.delaxes(axes[-1])

scores = []
for i, k in enumerate(range(2, 15)):
    # Обучение KMeans модели
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_embedded)
    
    # Рассчет silhouette score
    score = silhouette_score(X_embedded, kmeans.labels_)
    scores.append(score)
    # Визуализация
    cluster_centers_visual = np.array([X_embedded_visual[kmeans.labels_ == label].mean(axis=0) for label in np.unique(kmeans.labels_)])
    
    axes[i].scatter(X_embedded_visual[:, 0], X_embedded_visual[:, 1], c=kmeans.labels_, cmap='viridis')
    axes[i].scatter(cluster_centers_visual[:, 0], cluster_centers_visual[:, 1], c='red', marker='X')

    for center in cluster_centers_visual:
        circle = Circle(center, radius=0.2, fill=False, linestyle='-', linewidth=2, edgecolor='red')
        axes[i].add_artist(circle)
    
    axes[i].set_title(f'K = {k}, Silhouette Score = {score:.2f}')

print(scores)

optimal_cluster_k = scores.index(max(scores)) + 2

plt.tight_layout()
plt.show()


In [ ]:
from collections import Counter
kmeans = KMeans(n_clusters=7, n_init=10, random_state=42)
kmeans.fit(X_embedded_visual)
kmeans_df = df.copy()
kmeans_df['cluster_label'] = kmeans.labels_

Counter(kmeans.labels_)

In [ ]:
kmeans_df['censored_normalized'] # текст песни
kmeans_df['cluster_label'] # лейбл кластера
unique_clusters = sorted(kmeans_df['cluster_label'].unique())

In [ ]:
from collections import Counter
# Итерируемся по уникальным кластерам
cluster_text_top5_counters = {}

for cluster in unique_clusters:
    # Фильтруем строки по текущему кластеру и объединяем тексты в один большой список
    cluster_texts = kmeans_df[kmeans_df['cluster_label'] == cluster][f'{BASE_COLUMN}_normalized'].tolist()
    concatenated_texts = ' '.join(cluster_texts)

    # Считаем частоту каждого слова в данном кластере и отбираем топ 10
    counter = Counter(concatenated_texts.split())
    top_5_words = counter.most_common(5)

    cluster_text_top5_counters[cluster] = top_5_words

# Выводим результаты (в данном случае, они будут простыми из-за фиктивных данных)
cluster_text_top5_counters

Как мы можем интерпретировать? ->

Кластер 0: Содержит слова, которые можно ассоциировать с состоянием неопределенности или волнения ("стоить", "бояться", "черта", "ждать"). Есть также слова, указывающие на временную перспективу ("навсегда") и действия ("удалить").

Кластер 5: Содержит некоторые негативно окрашенные слова ("цензура", "страдать", "дрянный") и слова, связанные с планетарными или космическими темами ("планета").

Кластер 1: Смешивает английский и русский языки и содержит слова с сильной эмоциональной окраской ("цензура", "цензура", "you", "me").

Кластер 4: Включает слова, связанные с отношениями и эмоциями ("дать", "уйти", "любить", "ангел").

Кластер 3: Содержит слова, ассоциирующиеся с физиологией и эмоциональными состояниями ("любовь", "гормон", "кровь").

Кластер 6: Смешивает немецкий и русский языки и включает слова, связанные с желаниями и страхами ("хотеть", "бояться").

Кластер 2: Содержит слова, связанные с негативными эмоциями и действиями ("врать", "плакать", "бросить").

In [ ]:
kmeans_df[kmeans_df.cluster_label == 0].title[:5]

In [ ]:
import numpy as np
# Создаем пустой список, в который будем добавлять средние расстояния
neighbours = []

# Проходимся по каждой строке в исходном DataFrame
for i, v1 in enumerate(X_embedded):
    # Вычисляем расстояния от текущей точки до всех остальных
    distances = [np.linalg.norm(v1 - v2) for j, v2 in enumerate(X_embedded) if i != j]
    # Вычисляем среднее расстояние до 5 ближайших соседей
    neighbours.append(np.mean(sorted(distances)[:5]))

# Сортируем средние расстояния
neighbours = sorted(neighbours)

sns.lineplot(x=range(len(neighbours)), y=neighbours)
plt.axhline(y=np.mean(neighbours) + np.std(neighbours) / 2, color='g', linestyle='--', label="Mean + 0.5*STD")
plt.axhline(y=np.mean(neighbours), color='r', linestyle='--', label="Mean")
plt.axhline(y=np.mean(neighbours) - np.std(neighbours) / 2, color='g', linestyle='--', label="Mean - 0.5*STD")
plt.legend()
plt.title("Sorted Mean Distances to 5 Nearest Neighbours")
plt.xlabel("Index")
plt.ylabel("Mean Distance")
plt.show()

optimal_eps = np.mean(neighbours)
optimal_eps

In [ ]:
eps_metrics = [np.mean(neighbours) - np.std(neighbours) / 2, np.mean(neighbours), np.mean(neighbours) + np.std(neighbours) / 2]


In [ ]:
db_scores = []

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.ravel()

for i, eps in enumerate(eps_metrics):
    # Обучение DBSCAN модели
    dbsscan = DBSCAN(eps=eps, min_samples=2)
    dbsscan.fit(X_embedded)

    # Получение меток кластеров
    labels = dbsscan.labels_

    # Исключение шумовых точек для расчета центроида
    non_noise_indices = np.where(labels != -1)
    non_noise_labels = labels[non_noise_indices]
    non_noise_X = X_embedded[non_noise_indices]

    # Расчет центроидов для каждого кластера
    unique_labels = np.unique(non_noise_labels)
    cluster_centers = np.array([non_noise_X[non_noise_labels == label].mean(axis=0) for label in unique_labels])

    # Расчет Davies-Bouldin Index
    if len(unique_labels) > 1:  # DB index требует хотя бы два кластера
        db_score = davies_bouldin_score(non_noise_X, non_noise_labels)
        db_scores.append(db_score)
    else:
        db_scores.append(None)

    # Визуализация
    axes[i].scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels, cmap='viridis')
    if len(cluster_centers) > 0:  # Проверка на случай, если нет кластеров
        axes[i].scatter(cluster_centers[:, 0], cluster_centers[:, 1], c='red', marker='x')
    axes[i].set_title(f'eps = {round(eps, 4)}, Davies-Bouldin Score = {round(db_score, 4) if db_score is not None else "N/A"}')

plt.tight_layout()
plt.show()

In [ ]:
best_metric = np.mean(neighbours) - np.std(neighbours) / 2

dbsscan = DBSCAN(eps=best_metric, min_samples=2)
dbsscan.fit(X_embedded)
    
 # Получение меток кластеров
labels = dbsscan.labels_
dbscan_df = df.copy()
dbscan_df['cluster_label'] = dbsscan.labels_
Counter(labels)

In [ ]:
unique_clusters = sorted(dbscan_df['cluster_label'].unique())

cluster_text_top5_counters = {}

for cluster in unique_clusters:
    # Фильтруем строки по текущему кластеру и объединяем тексты в один большой список
    cluster_texts = dbscan_df[dbscan_df['cluster_label'] == cluster][f'{BASE_COLUMN}_normalized'].tolist()
    concatenated_texts = ' '.join(cluster_texts)

    # Считаем частоту каждого слова в данном кластере и отбираем топ 10
    counter = Counter(concatenated_texts.split())
    top_5_words = counter.most_common(5)
    
    cluster_text_top5_counters[cluster] = top_5_words

# Выводим результаты (в данном случае, они будут простыми из-за фиктивных данных)
cluster_text_top5_counters

Интерпретация кластеров

Кластер 0: Слова в этом кластере, такие как "хотеть", "бояться", "стоить", "знать", и некоторые немецкие слова, могут указывать на темы связанные с желаниями, страхами и знанием. Это может быть кластер, включающий слова, которые часто употребляются в контексте принятия решений или эмоциональных переживаний.

Кластер 1: Слова как "ести", "заваль", "нужный", "cash" предположительно связаны с потребностями и материальными аспектами.

Кластер 2: Слова на английском языке ("цензура", "цензура", "цензура") и русские нецензурные выражения ("цензура", "цензура") предположительно относятся к более жёсткому, возможно, конфликтному общению.

Кластер 3: Слова как "азиатка", "равно", "эгоэгоэгоист" могут указывать на какие-то культурные или психологические аспекты.

Кластер 4: Этот кластер может быть связан с женским полом или эмоциями ("девочка", "скакать", "мой").

Кластер 5: Слова вроде "дать", "уйти", "любить" могут быть связаны с межличностными отношениями.

Кластер 6: Слова как "гормон", "предвестник", "кровь" могут указывать на физиологические или медицинские аспекты.

Кластер 7: Этот кластер может быть связан с эмоциональными и физическими аспектами ("мой", "рука", "амбиция").

Кластер 8: Слова вроде "врать", "плакать", "цензура" могут указывать на отрицательные эмоции или конфликтные ситуации.

Кластер 9: Здесь много нецензурных или грубых слов, что может указывать на агрессивное или негативное общение.

Кластер 10: Слова как "цензура", "самый", "страдать" также предполагают негативные эмоции или отношения.

Кластер 11: Слова вроде "убить", "сердце", "шрам" могут быть связаны с темами потери, страдания или эмоционального воздействия.

Кластер 12: Этот кластер может иметь отношение к религиозным или духовным аспектам ("ангел").

-1: этот кластер содержит слова, которые не удалось чётко отнести к другим кластерам. Слова как "остановиться", "танцевать", "рассвет" могут указывать на разнообразные темы от эмоций до действий. (в контексте DBSCAN - кластер -1 - является тем кластером, куда ничто не вошло)